In [ ]:
import torch
import numpy as np
import cupy as cpy
from tqdm import tqdm
import eznf
from eznf import Tensor
from eznf import datasets
from eznf import optim

### 全连接神经网络

In [ ]:
class MLP(eznf.nn.Module):
    def __init__(self):
        super().__init__()
        self.networks = [
            eznf.nn.Linear(784, 256),
            eznf.nn.ReLU(),
            eznf.nn.Linear(256, 10)
        ]
    
    def forward(self, x):
        for i in self.networks:
            x = i(x)
        return x

### 卷积神经网络

In [ ]:
class CNN(eznf.nn.Module):
    def __init__(self):
        super().__init__()
        self.networks = [
            eznf.nn.Cov2d(1, 3, 3),
            eznf.nn.MaxPooling(2),
            eznf.nn.Flatten(),
            eznf.nn.Linear(507, 256),
            eznf.nn.ReLU(),
            eznf.nn.Linear(256, 10)
        ]
    
    def forward(self, x):
        for i in self.networks:
            x = i(x)
        return x

### 数据读取

In [ ]:
mnist = datasets.MNIST('./', False)
data = mnist.get()

### 数据预处理

In [ ]:
MLP_X_train, MLP_Y_train, MLP_X_test, MLP_Y_test = data

MLP_X_train = MLP_X_train / 255
MLP_X_train = np.vstack([i.flatten() for i in MLP_X_train.item])
MLP_X_test = MLP_X_test / 255
MLP_X_test = np.vstack([i.flatten() for i in MLP_X_test.item])

MLP_Y_train = eznf.one_hot(Tensor(MLP_Y_train), 10)
MLP_Y_test = eznf.one_hot(Tensor(MLP_Y_test), 10)

In [ ]:
CNN_X_train, CNN_Y_train, CNN_X_test, CNN_Y_test = data

CNN_X_train = CNN_X_train / 255
CNN_X_train = eznf.Tensor(CNN_X_train, requires_grad=False)
CNN_X_train = CNN_X_train[:,None,:,:]

CNN_X_test = CNN_X_test / 255
CNN_X_test = eznf.Tensor(CNN_X_test, requires_grad=False)
CNN_X_test = CNN_X_test[:,None,:,:]

CNN_Y_train = eznf.one_hot(Tensor(CNN_Y_train), 10)
CNN_Y_test = eznf.one_hot(Tensor(CNN_Y_test), 10)

### MLP训练

In [ ]:
epoches = 100
batch_size = 1024
steps = len(MLP_X_train) // batch_size

m = MLP()
loss = eznf.nn.CrossEntropyLoss()
optim = optim.SGD(alpha=0.01, model=m)

with tqdm(total=epoches) as t:
    for i in range(epoches):
        for j in range(steps):
            x = X_train[j*batch_size : (j+1)*batch_size]
            y = Y_train[j*batch_size : (j+1)*batch_size]
            out = m(x)
            l = loss(out, y.T) / batch_size
            l.backward()
            optim.step()
            optim.zero_grad()
        
        t.set_description('Epoch {}'.format(i), refresh=False)
        t.set_postfix(loss=l.item[0], refresh=False)
        t.update(1)

### CNN训练